# Исследование надёжности заёмщиков

## Загрузка и обзор данных

Импорт библиотек

In [45]:
import pandas as pd
from pymystem3 import Mystem
m = Mystem()
from collections import Counter

Функция для обзора и отображания информации о данных

In [2]:
def table_info (data):
    display(data.sample(10)) #10 случайных строк
    print('---------------')
    data.info() #инфо по столбцам
    print('---------------')
    print('Дубликатов:', data.duplicated().sum()) #кол-во дубликатов
    print('---------------')
    print('Пропущенных значений', data.isna().sum()) #кол-во пропущенных значений

Функция для приведения данных к едным заначениям в столбце с целями заема.

In [3]:
def rename_purpose(row):
    lem_purpose = m.lemmatize(row['purpose'])
    if 'автомобиль' in lem_purpose:
        return 'автомобиль'
    if ('жилье' in lem_purpose) or ('недвижимость' in lem_purpose ):
        return 'недвижимость'
    if 'свадьба' in lem_purpose:
        return 'свадьба'
    if 'образование' in lem_purpose:
        return 'образование'

Функцая для выделения категорый граждан с детьми и без.

In [4]:
def children_group(child):
    if child == 0:
        return 'без детей'
    if 1 <= child < 3:
        return 'есть дети'
    return 'многодетные'

Функция для разделения на группы по доходу.

In [5]:
def income_group(income):
    if income <= income_deb['total_income'].quantile(0.33):
        return 'низкий'
    if income_deb['total_income'].quantile(0.33) <= income < income_deb['total_income'].quantile(0.66):
        return 'средний'
    if income_deb['total_income'].quantile(0.66) <= income:
        return 'высокий'

Рассмотрим информация о таблице: 10 случайных строк,дубликаты, пропуски. 54 дубликата, пропущенные значения в столбцах days_employed и total_income. Пропуски в данных столбцах могут быть связаны с отсутвием заработка и трудового стажа. 

In [6]:
data=pd.read_csv('borrowers.csv')
table_info(data)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
1111,0,-5140.381842,48,среднее,1,вдовец / вдова,2,F,сотрудник,0,108365.942615,операции со своей недвижимостью
17047,1,NaN,28,высшее,0,женат / замужем,0,F,компаньон,0,NaN,свой автомобиль
5808,0,-98.900204,50,среднее,1,гражданский брак,1,M,компаньон,1,241915.373731,свадьба
13638,0,-9603.029157,50,Высшее,0,женат / замужем,0,M,сотрудник,0,147745.484928,покупка своего жилья
3710,0,-626.556056,42,среднее,1,женат / замужем,0,F,сотрудник,0,103143.302660,недвижимость
858,0,-2719.828190,41,среднее,1,женат / замужем,0,F,госслужащий,0,144115.000010,покупка своего жилья
10165,1,-2912.754151,30,среднее,1,женат / замужем,0,M,сотрудник,0,175778.533898,автомобиль
7149,0,-288.393090,50,среднее,1,женат / замужем,0,F,компаньон,0,114829.145442,автомобиль
4676,0,-1411.790052,58,среднее,1,в разводе,3,M,сотрудник,0,231730.748610,дополнительное образование
19884,0,-352.588094,30,среднее,1,в разводе,3,F,компаньон,0,304928.492307,строительство собственной недвижимости


---------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
---------------
Дубликатов: 54
---------------
Пропущенных значений children               0
days_employed       2174
dob_years              0
education   

Столбец с трудовым стажем содержит отрицательные значения.

In [7]:
data['days_employed'].describe()

count     19351.000000
mean      63046.497661
std      140827.311974
min      -18388.949901
25%       -2747.423625
50%       -1203.369529
75%        -291.095954
max      401755.400475
Name: days_employed, dtype: float64

Проверим не явные дубликаты в столбцах `family_status`, `children`, `purpose`.

In [8]:
print(data['family_status'].value_counts())

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64


В информации о количестве детей содержатся экстремальные значения: -1 и 20, скорее всего являются неявнями дубликатами.

In [9]:
data['children'].describe()

count    21525.000000
mean         0.538908
std          1.381587
min         -1.000000
25%          0.000000
50%          0.000000
75%          1.000000
max         20.000000
Name: children, dtype: float64

In [10]:
print(data['children'].value_counts())

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64


Цель для заема описана не стандартными фраза и в следствии мы имеем множество неявных дубликатов.

In [11]:
print(data['purpose'].value_counts())

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
покупка жилья для сдачи                   653
операции с жильем                         653
операции с коммерческой недвижимостью     651
покупка жилья                             647
жилье                                     647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
строительство недвижимости                620
покупка своего жилья                      620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

**Вывод**
1. В столбце "days_employed" - отрицательные значения. Стоит их перевести в положительные целые числа.
2. Пропущенные значения есть в столбцах "days_employed" и "total_income". Это 10% данных. Возможно связанно с отсутвием офицальной работы. Их заменим  медианой.
3. Столбец "children" содержит неявные дубликаты. Исправим их.
5. Нужно проводить лемматизицию значений в столбце "purpose".
6. В таблице 54 явных дубликата - после преобразований проверим их ещё раз.
 

## Шаг 2. Предобработка данных

### Обработка пропусков

In [12]:
display(data[data['days_employed'].isna()].head(5))


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


Переведем числа в столбце `days_employed` из отрицательных в положительные. И проверим результат.

In [13]:
data['days_employed'] = abs(data['days_employed'])

In [14]:
display(data.head(5)) 

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Заменим пропуски в `days_employed` на медиану.

In [15]:
data['days_employed'] = data['days_employed'].fillna(data['days_employed'].median())
print(data['days_employed'].isna().sum())

0


Проверим столбец `total_income` на наличие отрицательных чисел.

In [16]:
neg_num = 0
zero_num = 0
for i in data['total_income']:
    if i < 0:
        neg_num += 1
print(neg_num)

0


Заменяем пропуски в `total_income` на медану. Проверяем результат.

In [17]:
data['total_income'] = data['total_income'].fillna(data['total_income'].median())
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Замена типа данных

Меняем тип данных в столбцах "days_employed" и "total_income" с дробного на целый и проверяем. 

In [18]:
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   days_employed     21525 non-null  int64 
 2   dob_years         21525 non-null  int64 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int64 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int64 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int64 
 10  total_income      21525 non-null  int64 
 11  purpose           21525 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


### Обработка дубликатов

Заменим неявные дубликаты в `children`.

In [19]:
data['children'] = data['children'].replace(-1,1)
data['children'] = data['children'].replace(20,2)
print(data['children'].value_counts())

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64


В столбце `education` приведем все к нижнему регистру.

In [20]:
print(data['education'].value_counts())

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64


In [21]:
data['education'] = data['education'].str.lower()
print(data['education'].value_counts())

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64


**Вывод**
Явные дубликаты оставили в покое, а неявные заменили.

### Лемматизация

Лемматизируем значения в "purpose".

In [22]:
lemmas = []
for v in data['purpose']:
    lem = ' '.join(m.lemmatize(v))
    lemmas.append(lem)  

Посмотрим какие цели заема можно выделить.

In [23]:
print(Counter(lemmas)) 

Counter({'автомобиль \n': 973, 'свадьба \n': 797, 'на   проведение   свадьба \n': 777, 'сыграть   свадьба \n': 774, 'операция   с   недвижимость \n': 676, 'покупка   коммерческий   недвижимость \n': 664, 'операция   с   жилье \n': 653, 'покупка   жилье   для   сдача \n': 653, 'операция   с   коммерческий   недвижимость \n': 651, 'покупка   жилье \n': 647, 'жилье \n': 647, 'покупка   жилье   для   семья \n': 641, 'строительство   собственный   недвижимость \n': 635, 'недвижимость \n': 634, 'операция   со   свой   недвижимость \n': 630, 'строительство   жилой   недвижимость \n': 626, 'покупка   недвижимость \n': 624, 'строительство   недвижимость \n': 620, 'покупка   свой   жилье \n': 620, 'ремонт   жилье \n': 612, 'покупка   жилой   недвижимость \n': 607, 'на   покупка   свой   автомобиль \n': 505, 'заниматься   высокий   образование \n': 496, 'сделка   с   подержанный   автомобиль \n': 489, 'свой   автомобиль \n': 480, 'на   покупка   подержать   автомобиль \n': 479, 'на   покупка   ав

Применяем функцию для перезаписи цели заема и проверяем как она сработала.

In [24]:
data['purpose_category'] = data.apply(rename_purpose, axis=1)

In [25]:
display(data.sample())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category
8101,0,660,57,среднее,1,женат / замужем,0,M,компаньон,0,155317,покупка жилой недвижимости,недвижимость


### Повторная обработка дубликатов

In [26]:
print(data.duplicated().sum())

71


In [27]:
data = data.drop_duplicates().reset_index(drop=True)
print(data.duplicated().sum())

0


### Категоризация данных

- Дети и возврат кредита

Выделю в отдельную таблицу нужные столбцы. 3 категории: без детей, есть дети, многодетные (от 3 детей в семье). 

In [28]:
children_debt = data[['children','debt']]
print(children_debt['children'].value_counts())

0    14091
1     4855
2     2128
3      330
4       41
5        9
Name: children, dtype: int64


Добавляем столбец с категорией семьи по количеству детей.

In [44]:
children_debt['children_category'] = children_debt['children'].apply(children_group)
pd.set_option('mode.chained_assignment', None)

Проверяем.

In [30]:
display(children_debt['children_category'].value_counts())

без детей      14091
есть дети       6983
многодетные      380
Name: children_category, dtype: int64

- Семейное положение и возврат кредита

Выделяю в отдельную таблицу нужные столбцы. 

In [31]:
family_status_deb = data[['family_status','debt','family_status_id']]
print(family_status_deb['family_status'].value_counts())  

женат / замужем          12339
гражданский брак          4151
Не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64


- Цель кредита и возврат денег

Выделяю в отдельную таблицу нужные столбцы.

In [32]:
purpose_deb = data[['purpose_category','debt', 'purpose']]
display(purpose_deb['purpose_category'].value_counts())

недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2324
Name: purpose_category, dtype: int64

- Уровень дохода и возврат кредита

Выделяю в отдельную таблицу нужные столбцы

In [33]:
income_deb = data[['total_income','debt']]
display(income_deb.head())

,total_income,debt
0,253875,0
1,112080,0
2,145885,0
3,267628,0
4,158616,0


Разделим заемщиков на группы по квантилям.

In [35]:
income_deb['income_category'] = income_deb['total_income'].apply(income_group)
display(income_deb['income_category'].value_counts())
#pd.set_option('mode.chained_assignment', None)

высокий    7295
низкий     7080
средний    7079
Name: income_category, dtype: int64

**Вывод**
Данные поделенны на категории и готовы.

## Шаг 3. Поиск зависимости между категориями и возвратом заема в срок.

### Группы разделенные по количеству детей.

Сводная таблица по категории количество задолжностей в каждой группе.

In [36]:
children_debt_pivot = (
    children_debt.pivot_table(index=['children_category'], columns='debt', values='children', aggfunc='count')
    .reset_index()
)
display(children_debt_pivot)

debt,children_category,0,1
0,без детей,13028,1063
1,есть дети,6336,647
2,многодетные,349,31


Посчитаем процент задолжности в группах. 

In [37]:
children_debt_pivot['Процент должников'] = children_debt_pivot[1] / (children_debt_pivot[0]+children_debt_pivot[1]) * 100
children_debt_pivot['Процент должников'] = children_debt_pivot['Процент должников'].round(1)
display(children_debt_pivot)

debt,children_category,0,1,Процент должников
0,без детей,13028,1063,7.5
1,есть дети,6336,647,9.3
2,многодетные,349,31,8.2


**Вывод**
 Из данных в таблице можно заключить, что заёмщики с 1,2 детьми наименее надежные 9,3%. Самые надежные люди не имеющие детей 7,5%. 

### Группы выделенные по семейному положению.

Сводная таблица по интересующим данным.

In [38]:
family_status_deb_pivot = ( 
    family_status_deb.pivot_table(index=['family_status'], columns='debt', values ='family_status_id', aggfunc='count')
    .reset_index()
)
display(family_status_deb_pivot)

debt,family_status,0,1
0,Не женат / не замужем,2536,274
1,в разводе,1110,85
2,вдовец / вдова,896,63
3,гражданский брак,3763,388
4,женат / замужем,11408,931


Высчитываем процент должников в группах и округляем его.

In [39]:
family_status_deb_pivot['Процент должников'] = family_status_deb_pivot[1] /(family_status_deb_pivot[1] + family_status_deb_pivot[0])  * 100
family_status_deb_pivot['Процент должников'] = family_status_deb_pivot['Процент должников'].round(1)
display(family_status_deb_pivot)

debt,family_status,0,1,Процент должников
0,Не женат / не замужем,2536,274,9.8
1,в разводе,1110,85,7.1
2,вдовец / вдова,896,63,6.6
3,гражданский брак,3763,388,9.3
4,женат / замужем,11408,931,7.5


**Вывод**
Среди заёмщиков выделенных по семейному положению наибольший процент должников среди состоящих в гражданском браке
и не состоящих в отношениях 9,3% и 9,8% соответсвенно. Самые ответственные - граждане потерявшие супруга.

### Группы выделенные по уровню дохода.

Сводная таблица по данным.

In [40]:
income_deb_pivot = (
    income_deb.pivot_table(index=['income_category'], columns='debt', values ='total_income', aggfunc='count')
    .reset_index()
)
display(income_deb_pivot)

debt,income_category,0,1
0,высокий,6740,555
1,низкий,6502,578
2,средний,6471,608


Высчитываю процент должников.

In [41]:
income_deb_pivot['Процент должников'] = income_deb_pivot[1] / (income_deb_pivot[0] + income_deb_pivot[1]) * 100
income_deb_pivot['Процент должников'] = income_deb_pivot['Процент должников'].round(1)
display(income_deb_pivot)

debt,income_category,0,1,Процент должников
0,высокий,6740,555,7.6
1,низкий,6502,578,8.2
2,средний,6471,608,8.6


**Вывод**
    
Глядя на таблицу, можно сказать, что клиенты с средним доходом возвращают заем реже других групп граждан.

### Цель кредита.

Сводная таблица.

In [42]:
purpose_deb_pivot = (
    purpose_deb.pivot_table(index=['purpose_category'], columns='debt', values ='purpose', aggfunc='count')
    .reset_index()
)
display(purpose_deb_pivot)

debt,purpose_category,0,1
0,автомобиль,3903,403
1,недвижимость,10029,782
2,образование,3643,370
3,свадьба,2138,186


Находим процент должников.

In [43]:
purpose_deb_pivot['Процент должников'] = purpose_deb_pivot[1] / (purpose_deb_pivot[0] + purpose_deb_pivot[1]) * 100
purpose_deb_pivot['Процент должников'] = purpose_deb_pivot['Процент должников'].round(1)
display(purpose_deb_pivot)

debt,purpose_category,0,1,Процент должников
0,автомобиль,3903,403,9.4
1,недвижимость,10029,782,7.2
2,образование,3643,370,9.2
3,свадьба,2138,186,8.0


**Вывод**
Люди берущие кредит на покупку автомобиля или образование оказались наименее надежными 9,4% и 9,2%. В это же время заёмщики назвавшие своей целью покупку недвижимости являются более аккуратными в выплатах по кредиту.

## Шаг 4. Общий вывод

На выплату заема в срок влияют различные факторы.
Среди самых надежных заёмщиков можно выделить:
- в категориях выделенных по количеству детей, люди не имеющие детей оказались самыми прилежными платильщиками;
- среди граждан разного семейного статуса - потерявшие супруга;
- кредитополучатели со средним доходом чаще возвращали заём в срок;
- заёмщики бравшие деньги на операции с недвижимостью лучше остальных, взявших деньги на другие цели, выплачивали долг.